In [54]:
import psycopg2
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
import re
from __future__ import print_function

In [27]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

from nltk.stem.snowball import SnowballStemmer

In [2]:
Base = declarative_base()

In [3]:
class Recipe(Base):
    __tablename__ = 'recipes'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    url = Column(String, unique=True)
    calories = Column(Integer)
    fat = Column(Float)
    carbs = Column(Float)
    protein = Column(Float)
    cholesterol = Column(Float)
    sodium = Column(Float)
    servings = Column(Integer)
    #ingredients = 
    #__table_args__ = {'extend_existing': True}
    
    def __repr__(self):
        return "<Recipe(name='%s', url='%s')>" % (
            self.name, self.url)
    
class Ingredient(Base):
    __tablename__ = 'ingredients'
    id = Column(Integer, primary_key = True)
    ingredient = Column(String, nullable = False)
    recipe_id = Column(Integer, ForeignKey('recipes.id'))
    
    recipe = relationship(Recipe, back_populates = 'ingredients')
    #__table_args__ = {'extend_existing': True}
    def __repr__(self):
        return "<Ingredient(ingredient='%s')>" % self.ingredient

Recipe.ingredients = relationship("Ingredient", order_by=Ingredient.id, back_populates="recipe")

class Restaurant(Base):
    __tablename__ = 'restaurants'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    url = Column(String)
    zomatoID = Column(Integer, unique=True)
    costfortwo = Column(Float)
    featured_image = Column(String)
    photos = Column(String)
    menu_url = Column(String)
    price_range = Column(Integer)
    latitude = Column(Float)
    longitude = Column(Float)
    address = Column(String)
    
    def __repr__(self):
        return "<Restaurant(name='%s', url='%s')>" % (
            self.name, self.url)

class MenuItem(Base):
    __tablename__ = 'menuitems'
    id = Column(Integer, primary_key=True)
    menuitem = Column(String, nullable = False)
    description = Column(String)
    restaurant_id = Column(Integer, ForeignKey('restaurants.id'))
    price = Column(String)
    
    restaurant = relationship(Restaurant, back_populates = 'menuitems')
    __table_args__ = {'extend_existing': True}
    def __repr__(self):
        return "<MenuItem(item='%s', desc='%s')>" % (
            self.menuitem, self.description)
    
Restaurant.menuitems = relationship("MenuItem", order_by=MenuItem.id, back_populates="restaurant")

In [17]:
measurements = ("femtogram", "gigagram", "gram", "hectogram", "kilogram", \
                "long", "ton", "mcg", "megagram", "metric", "ton", "metric"\
                "tonne", "microgram", "milligram","nanogram", "ounce", \
                "lb", "oz", "each", "pound", "short", "Gram", "Ounce", "Pint", "Quart",\
                "Tablespoon", "Teaspoon", "Tablespoons", "Teaspoons", "Cups", "cup","Fluid Ounce", "fl oz", "Gallon", "Ounce", \
                "Pint", "Quart", "Tablespoon", "Teaspoon", "liter", "litre", "L", "ml", "fluid ounces", "can", "cans")

In [12]:
def find_measurement_words(i):
    ingredient_line = i
    for item in measurements:
        meas = str(" " + item.lower() + " ")
        a = re.search(meas, ingredient_line)
        if a > 0:
            unit = ingredient_line[a.start():a.end()].strip()
            quantity = ingredient_line[:a.start()].strip()
            ingredient = ingredient_line[a.end():].strip()
            break
        else:
            ilist = i.split(" ")
            quantity = ilist[0]
            unit = "each"
            ingredient = " ".join(ilist[1:])
    newitem = {"ingredient": ingredient,
    "unit": unit,
    "quantity":  quantity}
    try:
        fracsplit = ([float(k) for k in newitem["quantity"].split("/")])
        if len(fracsplit) >= 2:
            newitem["quantity"] = fracsplit[0] / fracsplit[1]
    except:
        None
    #print(newitem)
    return newitem

In [13]:
def clean_up_ingredient(ingredient_line):
    ingredient_line = re.sub("\[u\'", "", ingredient_line)
    ingredient_line = re.sub("\']", "", ingredient_line)
    return find_measurement_words(ingredient_line)

In [4]:
dbname = 'restaurants'
username = 'andylane'
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://andylane@localhost/restaurants


In [5]:
con = None
con = psycopg2.connect(database = "restaurants", user = "andylane")
Session = sessionmaker(bind=engine)
session = Session()

##Get recipe names into a vector

In [14]:
names = session.query(Recipe).all()

In [15]:
titles=[]
for item in names:
    titles.append(item.name)

##Get ingredient lists into vector, order matching names

In [18]:
synopses = []
for item in names:
    synopses.append(" ".join([clean_up_ingredient(a.ingredient)["ingredient"] for a in item.ingredients]))

In [19]:
calories=[]
for item in names:
    calories.append(item.calories)

http://brandonrose.org/clustering

In [23]:
stopwords = nltk.corpus.stopwords.words('english')
stemmer = SnowballStemmer("english")

## Match menu items to a cluster

In [ ]:
def tokenize_and_stem(title):
    stemmed_titles = []
    new_title=[]
    for word in nltk.word_tokenize(title):
        new_title.append(stemmer.stem(word))
    stemmed_titles.extend(new_title)
    return [i for i in stemmed_titles]

In [157]:
nltk.word_tokenize("braised short ribs")

['braised', 'short', 'ribs']

In [129]:
def cluster_recipes(titles, num_clusters = 380): 
    def tokenize_and_stem(title):
        stemmed_titles = []
        new_title=[]
        for word in nltk.word_tokenize(title):
            new_title.append(stemmer.stem(word))
        stemmed_titles.extend(new_title)
        return [i for i in stemmed_titles]

    def tokenize_only(title):
        tokenized_titles = []
        new_title=[]
        for word in nltk.word_tokenize(title):
            new_title.append(word)
        tokenized_titles.extend(new_title)
        return [i for i in tokenized_titles]

    #not super pythonic, no, not at all.
    #use extend so it's a big flat list of vocab
    totalvocab_stemmed = []
    totalvocab_tokenized = []
    for i in titles:
        totalvocab_stemmed.extend([j for j in tokenize_and_stem(i)])
        totalvocab_tokenized.extend([k for k in tokenize_only(i)])

    vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

    from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

    #define vectorizer parameters
    tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=200000,
                                     min_df=0.01, stop_words='english',
                                     use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

    count_vectorizer = CountVectorizer(max_df=0.9, max_features=200000,
                                     min_df=0.01, stop_words='english', tokenizer=tokenize_and_stem, ngram_range=(1,3))


    tfidf_matrix = tfidf_vectorizer.fit_transform(titles) #fit the vectorizer to synopses

    print(tfidf_matrix.shape)

    terms = tfidf_vectorizer.get_feature_names()

    from sklearn.metrics.pairwise import cosine_similarity
    dist = 1 - cosine_similarity(tfidf_matrix)

    from sklearn.cluster import KMeans
    #num_clusters = 380
    km = KMeans(n_clusters=num_clusters)
    km.fit(tfidf_matrix)
    clusters = km.labels_.tolist()
    
    from sklearn.externals import joblib
    #uncomment the below to save your model 
    #since I've already run my model I am loading from the pickle
    #joblib.dump(km,  'doc_cluster.pkl')
    #km = joblib.load('doc_cluster.pkl')
    recipes = {'title': titles, 'synopsis': synopses,'calories': calories, 'cluster': clusters}
    frame = pd.DataFrame(recipes, index = [clusters] , columns = ['title', 'cluster', 'calories'])
    frame['cluster'].value_counts()
    grouped = frame['calories'].groupby(frame['cluster']) #groupby cluster for aggregation purposes
    grouped.mean() #average rank (1 to 100) per cluster
    return {"cluster_designations": frame, "vectorizer": tfidf_vectorizer, "model": km}

## Assign description to a cluster

In [92]:
def cluster_ingredients(synopses, num_clusters = 100):
    '''
    Input: a list containing space-separated ingredients for a set of recipes.
    Output: a scikit-learn k-means model object 
    '''
    
    # here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed
    def tokenize_and_stem(text):
        # first tokenize by sentence, then by word to ensure that punctuation is caught as its own token
        tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
        filtered_tokens = []
        # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
        for token in tokens:
            if re.search('[a-zA-Z]', token):
                filtered_tokens.append(token)
        stems = [stemmer.stem(t) for t in filtered_tokens]
        return stems


    def tokenize_only(text):
        # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
        tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
        filtered_tokens = []
        # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
        for token in tokens:
            if re.search('[a-zA-Z]', token):
                filtered_tokens.append(token)
        return filtered_tokens

    #not super pythonic, no, not at all.
    #use extend so it's a big flat list of vocab
    totalvocab_stemmed = []
    totalvocab_tokenized = []
    for i in synopses:
        allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
        totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list

        allwords_tokenized = tokenize_only(i)
        totalvocab_tokenized.extend(allwords_tokenized)

    vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
    print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')
    print(vocab_frame.head())

    from sklearn.feature_extraction.text import TfidfVectorizer

    #define vectorizer parameters
    tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                     min_df=0.2, stop_words='english',
                                     use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

    tfidf_matrix = tfidf_vectorizer.fit_transform(synopses) #fit the vectorizer to synopses

    print(tfidf_matrix.shape)
    terms = tfidf_vectorizer.get_feature_names()

    from sklearn.metrics.pairwise import cosine_similarity
    dist = 1 - cosine_similarity(tfidf_matrix)

    from sklearn.cluster import KMeans
    #num_clusters = 100
    km = KMeans(n_clusters=num_clusters)
    km.fit(tfidf_matrix)
    clusters = km.labels_.tolist()

    from sklearn.externals import joblib
    #uncomment the below to save your model 
    #since I've already run my model I am loading from the pickle
    #joblib.dump(km,  'doc_cluster.pkl')
    #km = joblib.load('doc_cluster.pkl')
    recipes = {'title': titles, 'synopsis': synopses,'calories': calories, 'cluster': clusters}
    frame = pd.DataFrame(recipes, index = [clusters] , columns = ['title', 'cluster', 'calories'])
    frame['cluster'].value_counts()
    grouped = frame['calories'].groupby(frame['cluster']) #groupby cluster for aggregation purposes
    grouped.mean() #average rank (1 to 100) per cluster
    return {"cluster_designations": frame, "vectorizer": tfidf_vectorizer, "model": km}

In [135]:
recipe_clusters = cluster_recipes(titles)

(3941, 96)


In [136]:
ingredient_clusters = cluster_ingredients(synopses)

there are 117798 items in vocab_frame
                  words
milk               milk
white             white
vinegar         vinegar
all-purpos  all-purpose
flour             flour
(3941, 36)


In [124]:
X_new = recipe_clusters['vectorizer'].transform(["Soup"])
prediction = recipe_clusters['model'].predict(X_new)

In [125]:
# Predicts the cluster that a list of ingredients is in.
prediction

array([306], dtype=int32)

In [130]:
recipe_clusters["cluster_designations"].ix[306]['title']

KeyError: 306

In [158]:
def classify_menu_item(menu_item, menu_item_description, recipe_clusters, ingredient_clusters):
    menu_item = " ".join(tokenize_and_stem(menu_item))
    vectorized_menu_item = recipe_clusters["vectorizer"].transform([menu_item]) #can later pass a list of menu_items
    recipe_cluster = recipe_clusters["model"].predict(vectorized_menu_item)
    print(recipe_cluster)
    description = " ".join(tokenize_and_stem(description))
    #look up ingredient clusters list; return found ingredient clusters matching recipes
    # assign menu item description to an ingredient cluster
    # ask if there's a match 
    # for any 
    return recipe_clusters["cluster_designations"].ix[recipe_cluster]['title']

In [160]:
classify_menu_item("garlic pizza", "pizza smothered with garlic cloves and tomatoes", recipe_clusters, ingredient_clusters)

[112]


112                                   Garlic Pork Kabobs
112                        Skillet Garlic Chicken Dinner
112                         Roasted Garlic Chicken Penne
112                                 Lemon Garlic Chicken
112                        Garlic Caesar Chicken Tenders
112                        Garlic Cheese Chicken Rollups
112    Crispy Chicken Croquettes with Garlic Butter S...
112    Steamed Fresh Green Beans with Garlic Dill Hol...
112                                   Chicken and Garlic
112                                  Lime Garlic Chicken
112    Garlic Chicken Sausage and Summer Vegetable Saute
112                                 Garlic Chicken Pizza
112                          Garlic Chicken and Zucchini
112                  Chard Stalks and Garlic Scape Pasta
112                     Chicken with 40 Cloves of Garlic
112                    Chicken with 20 Cloves of Garlic 
Name: title, dtype: object

In [161]:
menu_items = session.query(MenuItem).filter(MenuItem.restaurant_id == 82).all()

In [199]:
[(index, item.description.encode('utf-8')) for index, item in enumerate(menu_items)]

[(0, 'almond milk - calabrian chili'),
 (1,
  'baby lettuces - pecan - pomegranate - preserved meyer lemon - fennel - sheep milk cheddar - apple cider vinaigrette'),
 (2,
  'roasted cauliflower - arugula - spiced pepita - scallion - pecorino - maple - sherry vinaigrette'),
 (3,
  "apple - celery root slaw - dill - avocado - leadbetter's english muffin - greens / fries"),
 (4,
  "fontina - mizuna- pickled red onion - sweet chili sauce - aioli - leadbetter's english muffin - greens / fries"),
 (5,
  'curried carrot slaw - cucumber & mint raita - torpedo roll - greens / fries'),
 (6,
  'housemade chickpea and mushroom patty - arugula - pickled jalape\xc3\xb1os - chipotle aioli - gruy\xc3\xa8re - acme roll - greens / fries'),
 (7,
  'arugula - pickled red onions - bread & butter pickles - acme roll - greens / fries'),
 (8, 'bacon - chili flake - spinach - grana padano'),
 (9, ''),
 (10, ''),
 (11, 'seven stills vodka, aqua di cedro, lemon verbana'),
 (12, 'blade gin, elderflower liqueur, g